# Part One: 
## Building a Web Farm for Developing and Testing

### Starting from scratch
We can see that we are now starting without any images

In [6]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE


### Download Centos 6 Image
We start by using the 'pull' command provided by docker to get the centos6:latest image

In [9]:
docker pull centos:centos6

centos6: Pulling from library/centos

Digest: sha256:1092df198d3da4faccc0660941b763ce5adf133b0ec71701b760d6f173c1f47b
Status: Downloaded newer image for centos:centos6


### Check image
Now that we downloaded the image we can check if the image downloaded correctly.

In [10]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos              centos6             8315978ceaaa        4 months ago        195 MB


### Launch an instance + install dependencies
We launch a docker instance from the centos:centos6 and install all dependencies.
Because the ipython notebook doesn't allow, in order to install all the dependencies, we had to run the docker container, and then use another terminal window to run "```docker attach [DOCKER_ID]```"to run all the commands manually there.
The commands that were run were:
``` bash
# Download wget
yum install wget

# Download + install epel
wget http://dl.fedoraproject.org/pub/epel/6/x86_64/epel-release-6-8.noarch.rpm
rpm -Uvh epel-release-6-8.noarch.rpm

# update
yum update

# install all dependencies
yum install which sudo httpd php openssh-server -y

# The we add to .bashrc
cd
echo "/sbin/service httpd start" >> .bashrc
echo "/sbin/service openssh-server start" >> .bashrc

# We copy our base image and exit
exit
```

In [5]:
docker run -i -t centos:centos6 /bin/bash

[root@548f70d44cb2 /]# yum install wget4cb2 /]# 
Loaded plugins: fastestmirror, ovl
Setting up Install Process
base                                                                                                            | 3.7 kB     00:00     
base/primary_db                                                                                                 | 4.7 MB     00:04     
extras                                                                                                          | 3.4 kB     00:00     
extras/primary_db                                                                                               |  37 kB     00:00     
updates                                                                                                         | 3.4 kB     00:00     
updates/primary_db                                                                                              | 5.4 MB     00:21     
Resolving Dependencies
--> Running transaction check
---> Package wget.x8

### Check nothing else is running

In [8]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


### Check that our container ID is there

In [11]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                     PORTS               NAMES
548f70d44cb2        centos:centos6      "/bin/bash"         12 minutes ago      Exited (0) 2 minutes ago                       reverent_pasteur


### Commit
We see that our container ID is ```548f70d44cb2```, so we want to commit that.
This will build a new container that has all the dependencies that we need, libraries, etc.

In [13]:
docker commit 548f70d44cb2 centos6:baseweb

sha256:f466708d751e2812b4ea181b91aeb44ca2efc031a015dc363f5bb900a60de0d5


### Check the images

In [14]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos6             baseweb             f466708d751e        35 seconds ago      373 MB
centos              centos6             8315978ceaaa        4 months ago        195 MB


### Start a container with new image
This time we will just start a container in daemon mode, and we will send the commands one by one with the docker exec command

In [16]:
docker run -d -i -t centos6:baseweb /bin/bash

cdb2b1a1a046652ec8bcf1ff4f80b0240d3a33dcf19da068d4a4d9d95507b226


##### Perfect! So now we can check what it's name is to refer to it through that

In [17]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
cdb2b1a1a046        centos6:baseweb     "/bin/bash"         34 seconds ago      Up 33 seconds                           dreamy_clarke


##### This container is called "dreamy_clarke" so we'll be able to refer to it through that name.

### Starting  a service
We first want to check what the commands and outputs have been so far using the **docker logs**

In [20]:
docker logs dreamy_clarke

Starting httpd: httpd: Could not reliably determine the server's fully qualified domain name, using 172.17.0.2 for ServerName
[  OK  ]
openssh-server: unrecognized service


##### We can see that openssh-server is an unrecognised service!

If we wanted to open ssh service we would run:

In [23]:
docker exec dreamy_clarke service sshd start

Generating SSH2 RSA host key: [  OK  ]
Generating SSH1 RSA host key: [  OK  ]
Generating SSH2 DSA host key: [  OK  ]
Starting sshd: [  OK  ]


**We now exit the container to fix this error in the original container**

In [24]:
docker stop dreamy_clarke

dreamy_clarke


In [25]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos6             baseweb             f466708d751e        12 minutes ago      373 MB
centos              centos6             8315978ceaaa        4 months ago        195 MB


### Remove image
We will now remove the image, but what will happen is that we'll run into a conflict

In [27]:
docker rmi centos6:baseweb

Error response from daemon: conflict: unable to remove repository reference "centos6:baseweb" (must force) - container cdb2b1a1a046 is using its referenced image f466708d751e


: 1

** In order to remove the image we need to first remove the container with the ID it is complaining**

In [28]:
docker rm cdb2b1a1a046

cdb2b1a1a046


In [29]:
docker rmi centos6:baseweb

Untagged: centos6:baseweb
Deleted: sha256:f466708d751e2812b4ea181b91aeb44ca2efc031a015dc363f5bb900a60de0d5
Deleted: sha256:a15c4e25591874dd8c9980c3312aa83f27a28055d90c391263bca2954314bdac


** Done!**

### Creating another image
We now don't have to re-run the image to create a brand new container to install the dependencies again, as we still have the container we created previously (the one we used to commit to the baseweb image).

We should re-start that container, and fix it!

In [33]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                      PORTS               NAMES
548f70d44cb2        centos:centos6      "/bin/bash"         32 minutes ago      Exited (0) 21 minutes ago                       reverent_pasteur


** We restart that container with the id 548f70d44cb2**

In [36]:
docker start 548f70d44cb2 

548f70d44cb2


**We now go to the .bashrc file and edit the last line**

In [57]:
# remove the last line of the .bashrc file
docker exec 548f70d44cb2 bash -c "cd && sed -i '$ d' .bashrc"

# Insert the correct line to run service
docker exec 548f70d44cb2 bash -c 'cd && echo "/sbin/service sshd start" >> .bashrc'

# Now we check the file
docker exec 548f70d44cb2 bash -c "cd && cat .bashrc"

# .bashrc

# User specific aliases and functions

alias rm='rm -i'
alias cp='cp -i'
alias mv='mv -i'

# Source global definitions
if [ -f /etc/bashrc ]; then
	. /etc/bashrc
fi

## Add services to start
/sbin/service httpd start
/sbin/service sshd start


### Sorted!
It seems that now it should be all good. So now we can stop the container, commit and we should be good.

In [58]:
docker stop 548f70d44cb2

548f70d44cb2


In [65]:
docker commit 548f70d44cb2 centos6:baseweb

sha256:34079efab3e4b83bd02553702d83d122a2d061d1d85f03d9b6a0d0ba7009af19


In [66]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos6             baseweb             34079efab3e4        2 seconds ago       373 MB
centos              centos6             8315978ceaaa        4 months ago        195 MB


In [67]:
docker run -d -t -i centos6:baseweb /bin/bash

695d1ddda23ecb2ee2aee71d944d9a57a36ca3d429f7b970d6ddc89135d71405


In [68]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
695d1ddda23e        centos6:baseweb     "/bin/bash"         22 seconds ago      Up 20 seconds                           fervent_wright


In [69]:
docker logs fervent_wright

Starting httpd: httpd: Could not reliably determine the server's fully qualified domain name, using 172.17.0.2 for ServerName
[  OK  ]
Generating SSH2 RSA host key: [  OK  ]
Generating SSH1 RSA host key: [  OK  ]
Generating SSH2 DSA host key: [  OK  ]
Starting sshd: [  OK  ]


### All good!
It seems that now all our servers are started as expected